In [89]:
from selenium import webdriver
import time
import requests 
import io
from PIL import Image
import hashlib
import os

DRIVER_PATH = r'C:\Users\Lenovo\Desktop\PhotosScrapper\chromedriver.exe'

In [90]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.rg_ic")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.irc_mi')
            for actual_image in actual_images:
                if actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            load_more_button = wd.find_element_by_css_selector(".ksb")
            if load_more_button:
                wd.execute_script("document.querySelector('.ksb').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [91]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [92]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=1):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [114]:
import pandas as pd
games = pd.read_csv(r'C:\Users\Lenovo\Desktop\GIT\GamersHub\WebScraping\export_dataframe2.csv',sep=';')

In [113]:
path = r'C:\Users\Lenovo\Desktop\GIT\GamersHub\WebScraping\PhotosScrapper\images'

folders = os.listdir(path)

for name in games['Titile']:
    name = name.replace(":", " ")
    name = name + " screenshot"
    folder_name = name.replace(" ", "_").lower()
    print(folder_name)
    if folder_name not in folders:
        search_term = name

        search_and_download(
            search_term=search_term,
            driver_path=DRIVER_PATH
        )

fallout_4_screenshot
divinity__original_sin_ii_-_definitive_edition_screenshot
outlast_2_screenshot
borderlands_3_screenshot
counter-strike__global_offensive_screenshot
league_of_legends_screenshot
citadel__forged_with_fire_screenshot
luigi's_mansion_3_screenshot
god_of_war_screenshot
the_witcher_2__assassins_of_kings_screenshot
minecraft_screenshot
batman__arkham_knight_screenshot
farming_simulator_17_screenshot
brawl_stars_screenshot
metro_exodus_screenshot
diablo_iii_screenshot
Found: 100 search results. Extracting links from 0:100
Found: 2 image links, done!
SUCCESS - saved https://images9.gry-online.pl/galeria/galeria_duze3/92318937.jpg - as ./images\diablo_iii_screenshot\8d75df290f.jpg
SUCCESS - saved https://i.ytimg.com/vi/K30D4YVcAo4/maxresdefault.jpg - as ./images\diablo_iii_screenshot\a24fdb9630.jpg
forza_horizon_4_screenshot
Found: 100 search results. Extracting links from 0:100
Found: 2 image links, done!
ERROR - Could not download https://www.gry-online.pl/galeria_duzee/40

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ksb"}
  (Session info: chrome=78.0.3904.97)
